# Análisis de la base de datos 
## Universidad de los Andes - Smurfit Westrock
### Poyecto Intermedio de Consultoría Empresarial (PICE) 202520
Daniel Benavides

This code performs an exploratory and preparatory analysis of Smurfit Westrock’s payment data. It begins by importing and cleaning raw datasets from Excel or CSV files, addressing missing values, duplicates, and inconsistencies. The data is then transformed through normalization of numerical variables and encoding of categorical ones such as suppliers, cost centers, and expense types. Exploratory Data Analysis (EDA) is conducted to visualize payment distributions, identify outliers and temporal trends, and examine correlations among key variables. Additionally, feature engineering is applied to create new indicators that capture behavioral patterns and transaction frequency, ensuring the dataset is ready for anomaly detection models. This analysis provides preliminary insights and recommendations to guide the development of Machine Learning models and improve overall data quality.

In [ ]:
# Data extraction libraries
import numpy as np
import pandas as pd

# Data visualizaton libraries 
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
import altair as alt
from prettytable import PrettyTable

from matplotlib import font_manager
plt.rcParams['font.family'] = 'Arial'

Data downloaded as Excel files

In [ ]:
# =================================================
# DEJAR COMENTADO SI YA SE TIENE EL DATAFRAME FINAL
# =================================================

# Read and concatenate Excel files
# df_excel = pd.concat([
#     pd.read_excel("PICE BD 2025-Parte 1.xlsx"),
#     pd.read_excel("PICE BD 2025-Parte 2.xlsx"),
#     pd.read_excel("PICE BD 2025-Parte 3.xlsx")
# ], ignore_index=True)
#
# columns_to_drop = [
#     "Número Documento Referencia", "Material", "Número de Cuenta",
#     "Acreedor", "Número Documento", "Descripción", "Documento Compras",
#     "Pos Docum Compras", "Año", "Activo Fijo", "Clase de Documento",
#     "Clase de Actividad", "Deudor", "Elemento PEP", "Orden", "Pedido Cliente"
# ]
#
# df_excel = (df_excel
#             .dropna(subset=["Número Documento Referencia"])
#             .drop(columns=columns_to_drop, errors='ignore'))
#
# df_excel

### PICE BD 2025 JOINT CSV FILE

Data downloaded as CSV file (ideal)

In [ ]:
# df_csv = df_excel.to_csv("PICE BD 2025.csv", index=False)
df_csv = pd.read_csv("PICE BD 2025.csv", low_memory=False)
df_csv.info()

df_csv.head()

# 1. Data Cleaning and Transformation

In [ ]:
db = df_csv.copy()
db.head()

In [ ]:
db.rename(columns={"En moneda de la sociedad": "Monto transado",
                   "Período": "Mes",
                   " Año ": "Año",
                   "Se ha anulado el Documento": "Estatus de anulación"}, inplace=True)

db.head()

In [ ]:
def clean_numeric_series(s: pd.Series) -> pd.Series:
    s = s.astype(str)
    s = s.str.replace(r'[^\d\-\.,]', '', regex=True)
    s = s.str.replace(',', '', regex=False)
    return pd.to_numeric(s, errors='coerce')

# En el dataframe, existen registro anulados marcados ("X") y registros no anulados (NaN)
# Transformar "X" a 1 y NaN a 0 en la columna "Estatus de anulación"
db["Estatus de anulación"] = db["Estatus de anulación"].apply(lambda x: 1 if x == "X" else 0)

# Transform date columns
# db["Fecha Entrada"] = pd.to_datetime(db["Fecha Entrada"], errors='coerce')
# db["Año Entrada"] = db["Fecha Entrada"].dt.year.round(0).astype('Int64')
# db["Mes Entrada"] = db["Fecha Entrada"].dt.month.round(0).astype('Int64')
# db["Dia Entrada"] = db["Fecha Entrada"].dt.day.round(0).astype('Int64')

# db["Fecha Valor"] = pd.to_datetime(db["Fecha Valor"], errors='coerce', dayfirst=True)
# db["Año Valor"] = db["Fecha Valor"].dt.year.round(0).astype('Int64')
# db["Mes Valor"] = db["Fecha Valor"].dt.month.round(0).astype('Int64')
# db["Dia Valor"] = db["Fecha Valor"].dt.day.round(0).astype('Int64')

db.head()

### Relevant variables

In [ ]:
variables = ["Denominación","Centro de Coste", "Usuario", "Período", 
             "Clase", "Clase de Movimiento V", "Tipo de Documento", "Centro de Beneficio", 
             "Clase de Factura", "División", "Se ha anulado el Documento",
             "Sector"]

db[variables].nunique().sort_values(ascending=False)

table = PrettyTable()
table.field_names = ["Variable", "Unique Values"]
for var, unique_count in db[variables].nunique().sort_values(ascending=False).items():
    table.add_row([var, unique_count])
print(table)

In [ ]:
# Count percentage of missing values per column
missing_percentage = db.isnull().mean() * 100
missing_percentage = missing_percentage[missing_percentage > 0].sort_values(ascending=False)
round(missing_percentage, 2)

table = PrettyTable()
table.field_names = ["Variable", "Missing Percentage"]
for var, perc in missing_percentage.items():
    table.add_row([var, f"{perc:.2f}%"])
print(table)


In [ ]:
variables_to_impute = [ "Sector", "Clase de Factura", "Clase de Movimiento V",
                        "Centro de Coste", "Fecha Valor", "Ledger", "Cantidad",
                        "Centro", "Hora", "Clase", "División" ]

# 2. Missing Data Analysis
The missing data analysis corresponds to the reuslts provided in the python file **Missing_Data_Analysis**

In [ ]:
from Missing_Data_Analysis import comprehensive_missingness_analysis

results = comprehensive_missingness_analysis(db, variables_to_impute)

# 3. KNN Imputation
The K-Nearest Neighbors (KNN) imputation method is a technique used to fill in missing values in a dataset by leveraging the K-Nearest Neighbors algorithm. This method estimates missing values based on the values of the "k" most similar data points (neighbors) in the dataset. 

In [ ]:
# KNN Imputation



# 4. Analysis of Cleaned and Imputed Data

In [ ]:
# Heatmap of number of transactions by day and month
# db_heatmap = db.pivot_table(index="Mes Entrada", columns="Dia Entrada", values="Monto (Millones)", aggfunc="count", fill_value=0)
# db_heatmap.index = db_heatmap.index.astype(int)
# db_heatmap.columns = db_heatmap.columns.astype(int)

# plt.figure(figsize=(12, 8))
# sns.heatmap(db_heatmap, cmap="YlGnBu", annot=True, fmt="d")
# plt.title("Transacciones (Entrada) diarias")

In [ ]:
# Heatmap of number of transactions by day and month
# db_heatmap = db.pivot_table(index="Mes Valor", columns="Dia Valor", values="Monto (Millones)", aggfunc="count", fill_value=0)
# db_heatmap.index = db_heatmap.index.astype(int)
# db_heatmap.columns = db_heatmap.columns.astype(int)

# plt.figure(figsize=(12, 8))
# sns.heatmap(db_heatmap, cmap="YlGnBu", annot=True, fmt="d")
# plt.title("Transacciones (Valor) diarias")

In [ ]:
pio.templates["plotly"].layout.font.family = "Arial"
pio.templates["plotly_white"].layout.font.family = "Arial"
pio.templates.default = "plotly"

In [ ]:
# Monto (millones) transado por Centro de Coste
# db_cc_ano = db.groupby(["Centro de Coste"])["Monto (Millones)"].sum().reset_index()

# px.bar(db_cc_ano,
       # x="Centro de Coste",
       # y="Monto (Millones)",
       # title="Monto (Millones) transado por Centro de Coste")

In [ ]:
# Promedio Monto (millones) transado por Centro de Coste
# db_cc_ano = db.groupby(["Centro de Coste"])["Monto (Millones)"].mean().reset_index()

# px.bar(db_cc_ano,
       # x="Centro de Coste",
       # y="Monto (Millones)",
       # title="Promedio del Monto (Millones) transado por Centro de Coste")